In [1]:
import pandas as pd
import numpy as np

import plotly.offline as po
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.neighbors import LocalOutlierFactor
from scipy.signal import butter, filtfilt
from pykalman import KalmanFilter

In [2]:
# Get data
data = pd.read_csv("Ezra Data.csv")
Hz = 100

In [3]:
# Determine outliers through Local Outlier Factor
measures = data.iloc[:, 1:-1]
model = LocalOutlierFactor()
model.fit(measures)
outliers = model.fit_predict(measures) == -1

# What percentage are outliers?
print(sum(outliers) / len(measures))

0.040284584980237154


In [17]:
# Make figure to show outliers in both accelerometer and gyroscope
fig = make_subplots(rows = 3, cols = 1)

fig.add_trace(go.Scatter(
    x = data["Time (ms)"].loc[~outliers].loc[::3],
    y = data["Accel Y (m/s^2)"].loc[~outliers].loc[::3],
    marker_color = "blue",
    mode = "markers",
    hoverinfo = "skip",
    showlegend = False
), col = 1, row = 1)

fig.add_trace(go.Scatter(
    x = data["Time (ms)"].loc[outliers],
    y = data["Accel Y (m/s^2)"].loc[outliers],
    marker_color = "orange",
    mode = "markers",
    hoverinfo = "skip",
    showlegend = False
), col = 1, row = 1)

fig.add_trace(go.Scatter(
    x = data["Time (ms)"].loc[~outliers].loc[::3],
    y = data["Gyro Y (rad/s)"].loc[~outliers].loc[::3],
    marker_color = "blue",
    mode = "markers",
    hoverinfo = "skip",
    showlegend = False
), col = 1, row = 2)

fig.add_trace(go.Scatter(
    x = data["Time (ms)"].loc[outliers],
    y = data["Gyro Y (rad/s)"].loc[outliers],
    marker_color = "orange",
    mode = "markers",
    hoverinfo = "skip",
    showlegend = False
), col = 1, row = 2)

fig.add_trace(go.Scatter(
    x = data["Time (ms)"].loc[~outliers].loc[::3],
    y = data["Lin Accel Y (m/s^2)"].loc[~outliers].loc[::3],
    marker_color = "blue",
    mode = "markers",
    hoverinfo = "skip",
    showlegend = False
), col = 1, row = 3)

fig.add_trace(go.Scatter(
    x = data["Time (ms)"].loc[outliers],
    y = data["Lin Accel Y (m/s^2)"].loc[outliers],
    marker_color = "orange",
    mode = "markers",
    hoverinfo = "skip",
    showlegend = False
), col = 1, row = 3)

fig.update_layout(
    title = "Outliers detected by Local Outlief Factor",
    xaxis_title = "",
    yaxis_title = "Accel (m/s^2)",
    xaxis2_title = "",
    yaxis2_title = "Gyro (rad/s)",
    xaxis3_title = "Time (ms)",
    yaxis3_title = "Lin Accel (m/s^2)",
    margin = {"t": 40, "b": 40, "l": 40, "r": 40},
    width = 1000, height = 400,
    hovermode = False
)

po.offline.plot(fig, filename = "outliers.html")

'outliers.html'

In [5]:
# Impute detected outliers with interpolation
data.iloc[outliers, 1:-1] = np.nan
data = data.interpolate()

In [6]:
def lowpassFilter(data, cutoff, fs, order = 5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype = 'low', analog = False)
    y = filtfilt(b, a, data)
    return y

# Copy data for comparison
filterData = data.copy()

# Apply filter for gyroscope
cutoff = 2
filterData.iloc[:, 1:4] = filterData.iloc[:, 1:4].apply(lowpassFilter, args = (cutoff, Hz))

# Apply filter for accelerometer
cutoff = 1
filterData.iloc[:, 4:7] = filterData.iloc[:, 4:7].apply(lowpassFilter, args = (cutoff, Hz))

# Apply filter for linear accelerometer
cutoff = 5
filterData.iloc[:, 7:-1] = filterData.iloc[:, 7:-1].apply(lowpassFilter, args = (cutoff, Hz))

In [16]:
# Make figure to show outliers in both accelerometer and gyroscope
fig = make_subplots(rows = 3, cols = 1)

fig.add_trace(go.Scatter(
    x = data["Time (ms)"],
    y = data["Accel Y (m/s^2)"],
    marker_color = "blue",
    mode = "lines",
    line_width = 5,
    hoverinfo = "skip",
    showlegend = False
), col = 1, row = 1)

fig.add_trace(go.Scatter(
    x = filterData["Time (ms)"],
    y = filterData["Accel Y (m/s^2)"],
    marker_color = "orange",
    mode = "lines",
    line_width = 5,
    hoverinfo = "skip",
    showlegend = False
), col = 1, row = 1)

fig.add_trace(go.Scatter(
    x = data["Time (ms)"],
    y = data["Gyro Y (rad/s)"],
    marker_color = "blue",
    mode = "lines",
    line_width = 5,
    hoverinfo = "skip",
    showlegend = False
), col = 1, row = 2)

fig.add_trace(go.Scatter(
    x = filterData["Time (ms)"],
    y = filterData["Gyro Y (rad/s)"],
    marker_color = "orange",
    mode = "lines",
    line_width = 5,
    hoverinfo = "skip",
    showlegend = False
), col = 1, row = 2)

fig.add_trace(go.Scatter(
    x = data["Time (ms)"],
    y = data["Lin Accel Y (m/s^2)"],
    marker_color = "blue",
    mode = "lines",
    line_width = 5,
    hoverinfo = "skip",
    showlegend = False,
), col = 1, row = 3)

fig.add_trace(go.Scatter(
    x = filterData["Time (ms)"],
    y = filterData["Lin Accel Y (m/s^2)"],
    marker_color = "orange",
    mode = "lines",
    line_width = 5,
    hoverinfo = "skip",
    showlegend = False,
), col = 1, row = 3)

fig.update_layout(
    title = "Cleaned data compared to lowpass filtered data in Y direction",
    xaxis_title = "",
    yaxis_title = "Accel (m/s^2)",
    xaxis2_title = "",
    yaxis2_title = "Gyro (rad/s)",
    xaxis3_title = "Time (ms)",
    yaxis3_title = "Lin Accel (m/s^2)",
    margin = {"t": 40, "b": 40, "l": 40, "r": 40},
    width = 1000, height = 400,
    hovermode = False
)

po.offline.plot(fig, filename = "lowpass_filter.html")

'lowpass_filter.html'

In [8]:
# Get temporal sensor data
measures = filterData.drop(columns = ["Time (ms)", "Label"])

# Define initial state
initialMeans = np.zeros(measures.shape[1])

# Define state transition matrix and observation matrix
transMatrix = np.eye(measures.shape[1])
obsMatrix = np.eye(measures.shape[1])

# Initialize the Kalman filter
kf = KalmanFilter(
    transition_matrices = transMatrix,
    observation_matrices = obsMatrix,
    initial_state_mean = initialMeans
)

# Mask invalid values
maskedMeasures = np.ma.masked_invalid(measures)

# Estimate parameters
kf = kf.em(maskedMeasures, n_iter = 5)

# Apply the filter to the data
filteredMeans, filteredCov = kf.filter(maskedMeasures)

# Add filtered data to the original data table
kalmanData = filterData.copy()
kalmanData.iloc[:, 1:-1] = filteredMeans

In [21]:
# Make figure to show outliers in both accelerometer and gyroscope
fig = make_subplots(rows = 3, cols = 1)

fig.add_trace(go.Scatter(
    x = data["Time (ms)"],
    y = data["Accel Y (m/s^2)"],
    marker_color = "blue",
    mode = "lines",
    line_width = 5,
    hoverinfo = "skip",
    showlegend = False
), col = 1, row = 1)

fig.add_trace(go.Scatter(
    x = kalmanData["Time (ms)"],
    y = kalmanData["Accel Y (m/s^2)"],
    marker_color = "orange",
    mode = "lines",
    line_width = 5,
    hoverinfo = "skip",
    showlegend = False
), col = 1, row = 1)

fig.add_trace(go.Scatter(
    x = data["Time (ms)"],
    y = data["Gyro Y (rad/s)"],
    marker_color = "blue",
    mode = "lines",
    line_width = 5,
    hoverinfo = "skip",
    showlegend = False
), col = 1, row = 2)

fig.add_trace(go.Scatter(
    x = kalmanData["Time (ms)"],
    y = kalmanData["Gyro Y (rad/s)"],
    marker_color = "orange",
    mode = "lines",
    line_width = 5,
    hoverinfo = "skip",
    showlegend = False
), col = 1, row = 2)

fig.add_trace(go.Scatter(
    x = data["Time (ms)"],
    y = data["Lin Accel Y (m/s^2)"],
    marker_color = "blue",
    mode = "lines",
    line_width = 5,
    hoverinfo = "skip",
    showlegend = False,
), col = 1, row = 3)

fig.add_trace(go.Scatter(
    x = kalmanData["Time (ms)"],
    y = kalmanData["Lin Accel Y (m/s^2)"],
    marker_color = "orange",
    mode = "lines",
    line_width = 5,
    hoverinfo = "skip",
    showlegend = False,
), col = 1, row = 3)

fig.update_layout(
    title = "Cleaned data compared to Kalman filtered data after lowpass in Y direction",
    xaxis_title = "",
    yaxis_title = "Accel (m/s^2)",
    xaxis2_title = "",
    yaxis2_title = "Gyro (rad/s)",
    xaxis3_title = "Time (ms)",
    yaxis3_title = "Lin Accel (m/s^2)",
    margin = {"t": 40, "b": 40, "l": 40, "r": 40},
    width = 1000, height = 400,
    hovermode = False
)

po.offline.plot(fig, filename = "kalman_filter.html")

'kalman_filter.html'